In [1]:
from project.retinanet.model import resnet34
from torchvision.transforms import Compose
from torchvision.transforms import ToPILImage
from project.datasetLoader.dataloader import EstimatedDeepFish,Resizer,Augmenter,Normalizer,Permuter, ToTensor
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import collections
import torch.optim as optim
import numpy as np
from tqdm import tqdm

In [2]:
class MultiEpochsDataLoader(torch.utils.data.DataLoader):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._DataLoader__initialized = False
        self.batch_sampler = _RepeatSampler(self.batch_sampler)
        self._DataLoader__initialized = True
        self.iterator = super().__iter__()

    def __len__(self):
        return len(self.batch_sampler.sampler)

    def __iter__(self):
        for i in range(len(self)):
            yield next(self.iterator)


class _RepeatSampler(object):
    """ Sampler that repeats forever.

    Args:
        sampler (Sampler)
    """

    def __init__(self, sampler):
        self.sampler = sampler

    def __iter__(self):
        while True:
            yield from iter(self.sampler)

In [3]:
from torchvision.transforms import ToTensor, Normalize, ToPILImage

DATASET_PATH = "./DATASET/"

dataset= EstimatedDeepFish('./annotations.csv',
                           DATASET_PATH, 
                           Compose([
                               Normalize(
                                   mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225],
                               ),
                               Resizer(480, 480),  
                            #    Augmenter(),
                            #    Resizer(480, 480),
                            #    Permuter(),
                            ]))
dataloader= MultiEpochsDataLoader(dataset, 64, num_workers=1, shuffle=True)

for i in tqdm(dataloader):
    pass

print("DONE")

305 rows were removed from image dataset!
3751
Index(['file', 'bbox', 'class', 'size (cm)'], dtype='object')


  0%|          | 0/9 [00:00<?, ?it/s]


For getting from Loader: [0.01] secondsFor getting from Loader: [0.00] secondsFor getting from Loader: [0.00] seconds

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/muhammadtarek/Work/Torpedo/ML/uwrc/venv/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/muhammadtarek/Work/Torpedo/ML/uwrc/venv/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/muhammadtarek/Work/Torpedo/ML/uwrc/venv/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/Users/muhammadtarek/Work/Torpedo/ML/uwrc/project/datasetLoader/dataloader.py", line 46, in __getitem__
    sample = self.transform(sample)
             ^^^^^^^^^^^^^^^

In [ ]:
dataset.num_classes()[0]

In [ ]:
class PipelineModel(nn.Module):
    def __init__(self,num_classes):
        super().__init__()
        self.detector= resnet34(num_classes=num_classes,pretrained=True)

    def forward(self,x):
        return self.detector((x['img'],x['annot'],x['number']))
model= PipelineModel(len(dataset.num_classes()[0]))

from torchinfo import summary
summary(model)